In [7]:
import random

import numpy as np
import sympy as sp
import matplotlib.pyplot as plt


class Function:
    def __init__(self, value, delt=0.001):
        self.value = value

        def grad(x):
            array = []
            for i in range(len(x)):
                x[i] += delt
                first_val = value(x)
                x[i] -= 2 * delt
                second_val = value(x)
                x[i] += delt
                array.append((first_val - second_val) / (2 * delt))
            return np.array(array)

        self.grad = grad


def grad_desc(func, lr, x, eps):
    points = [x]

    while True:
        prev_x = x
        x = x - lr(Function(lambda a: func.value(x - a * func.grad(x)))) * func.grad(x)
        points.append(x)
        if abs(func.value(x) - func.value(prev_x)) < eps:
            break

    return np.array(points)


def right_border_calc(func):
    right_start = 0.25
    zero = func.value(0)
    while zero >= func.value(right_start):
        right_start *= 2

    return right_start


def dichotomy(func, a_1, a_2, eps, delt):
    while abs(a_1 - a_2) >= eps:
        new_a_1 = (a_1 + a_2) / 2 - delt
        new_a_2 = (a_1 + a_2) / 2 + delt

        if func.value(new_a_2) > func.value(new_a_1):
            a_2 = new_a_2
        else:
            a_1 = new_a_1

    return (a_1 + a_2) / 2


def func_generator(n, k):
    vals = [random.uniform(1.0, k) for _ in range(n)]
    vals.sort()
    vals[0] = 1
    vals[n - 1] = k
    q, r = np.linalg.qr(np.random.rand(n, n))
    matr = np.matmul(np.matmul(q, np.diag(vals)), np.transpose(q))
    return matr


class FunctionCalculator:
    def __init__(self, matr):
        self.matr = matr

        def calculate(vect):
            return sum(matr[i][j] * vect[i] * vect[j] for i in range(len(vect)) for j in range(len(vect)))

        self.calculate = calculate

In [8]:

def func(x):
    return (x[0] - 4) ** 2 / 10 + (5 - x[1]) ** 2


def lr_wrapper(eps, delt):
    return lambda func: dichotomy(func, 0, right_border_calc(func), eps, delt)


print(grad_desc(Function(FunctionCalculator(func_generator(4, 5)).calculate), lr_wrapper(0.01, 0.0002),
                np.array((20., 20., 20., 20.)), 0.001))

[[2.00000000e+01 2.00000000e+01 2.00000000e+01 2.00000000e+01]
 [8.14413770e+00 1.30828320e+01 1.56999241e+01 2.18264776e+00]
 [4.49023799e+00 9.25812440e+00 9.24992343e+00 7.61104985e+00]
 [2.75347168e+00 5.02910294e+00 6.13542045e+00 5.19420737e-01]
 [1.49679689e+00 3.51683778e+00 3.50489049e+00 2.93335365e+00]
 [1.10345412e+00 2.05692880e+00 2.46943232e+00 3.51650277e-01]
 [5.95450293e-01 1.41337106e+00 1.39822636e+00 1.21679556e+00]
 [4.63650596e-01 8.72973641e-01 1.03958707e+00 1.77299725e-01]
 [2.55575121e-01 6.02305886e-01 5.99246257e-01 5.07658666e-01]
 [1.92692533e-01 3.61851573e-01 4.31852645e-01 7.04613991e-02]
 [1.05629277e-01 2.49394910e-01 2.47772344e-01 2.11382090e-01]
 [8.00878777e-02 1.49992916e-01 1.79402512e-01 2.79261737e-02]
 [4.36546051e-02 1.03266128e-01 1.02442310e-01 8.80234926e-02]
 [3.36651262e-02 6.36681279e-02 7.55460547e-02 1.38316162e-02]
 [1.78793620e-02 4.28846315e-02 4.20844700e-02 3.80501739e-02]
 [1.46059736e-02 2.78629128e-02 3.28283455e-02 6.812763